In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.impute import SimpleImputer
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [ ]:
data=pd.read_csv('/kaggle/input/building-permit-applications-data/Building_Permits.csv')

In [ ]:
data

In [ ]:
#getting the sum and percentage values having null 
total_missing_values=data.isnull().sum().sort_values(ascending=False)
percent_missing_values= (total_missing_values/len(data))*100
missing_data_percentage=pd.concat([total_missing_values,percent_missing_values],axis=1,keys=['total','percent'])
data.drop(list(missing_data_percentage[missing_data_percentage.percent>90].index),axis=1,inplace=True)

In [ ]:
data

In [ ]:
#getting only numbered column
number_column=data.select_dtypes(include=['int64','float64']).columns
impute=SimpleImputer(strategy="median")
data[number_column]=impute.fit_transform(data[number_column])



In [ ]:

categorial_columns=[i for i in data.columns if (data[i].dtype=='O') and (data[i].isnull().sum()>=1)]

In [ ]:
for i in categorial_columns:
    word=data[i].mode()
    data[i].fillna(word[0],inplace=True)

In [ ]:
data

In [ ]:
des=data.describe()

In [ ]:
des

In [ ]:
plt.figure()
for i in range(len(number_column)):
    plt.figure(i)
    sns.boxplot(x=data[number_column[i]])
    plt.xlabel(number_column[i])

In [ ]:
#getting all outliers
outlier={}
def out(i):
    iqr=des.loc["75%",i]-des.loc['25%',i]
    lower_limit=des.loc['25%',i]-(1.5*iqr)
    heigher_limit=des.loc['75%',i]+(1.5*iqr)
    outlier[i]=[lower_limit,heigher_limit]
for i in range(len(number_column)):
    out(number_column[i])
        


In [ ]:
outlier

In [ ]:
for i in outlier.items():
    char, num=list(i)
    #print(char,'->',num)
    if num[0]!=num[1]:
        data=data[(data[char]>num[0])&(data[char]<num[1])]
    else:
        data=data[data[char]==num[0]]

In [ ]:
data

In [ ]:
plt.figure()
for num,i in enumerate(number_column):
    plt.figure(num)
    sns.boxplot(x=data[i])
    plt.xlabel(i)

In [ ]:
more_outlier=['Estimated Cost','Revised Cost']
more=data[more_outlier]
data_2=more.describe()
data_2

In [ ]:
outlier2={}
def out_2(col):
    for i in col:
        iqr=data_2.loc['75%',i]-data_2.loc['25%',i]
        l_bound=data_2.loc['25%',i]-(1.5*iqr)
        h_bound=data_2.loc['75%',i]+(1.5*iqr)
        outlier2[i]=[l_bound,h_bound]
    return outlier2
out=out_2(more_outlier)
out

In [ ]:
for i in out.items():
    obj, num=list(i)
    data=data[(data[obj]>num[0])&(data[obj]<num[1])]

In [ ]:
data

In [ ]:
plt.figure()
for num,i in enumerate(number_column):
    plt.figure(num)
    sns.boxplot(x=data[i])
    plt.xlabel(i)

In [ ]:
data.describe()

In [ ]:
# from here we are scaling and normalizing the data
data_norm=data.copy()
data_scale=data.copy()

In [ ]:
minmax=MinMaxScaler()
data_norm[number_column]=minmax.fit_transform(data_norm[number_column])

In [ ]:
scaling=StandardScaler()
data_scale[number_column]=scaling.fit_transform(data_scale[number_column])

In [ ]:
data_scale

In [ ]:
data.columns

In [ ]:
col=list(data.columns)

In [ ]:
date=[i for i in col if 'Date' in i]

In [ ]:
#before all date types were object here we are converting it to datetime datatype
for i in date:
    
    data[i]=pd.to_datetime(data[i],infer_datetime_format=True)

In [ ]:
data

In [ ]:
for i in date:
    print(data[i].dtype,'\n')